In [1]:
import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
import pickle

import numpy as np



In [2]:
import sys
sys.path.insert(0, '..')


import System as SEQ
%run flow_EQS.ipynb


---------------------- SCIANN 0.6.4.5 ---------------------- 
For details, check out our review paper and the documentation at: 
 +  "https://www.sciencedirect.com/science/article/pii/S0045782520307374", 
 +  "https://arxiv.org/abs/2005.08803", 
 +  "https://www.sciann.com". 

 Need support or would like to contribute, please join sciann`s slack group: 
 +  "https://join.slack.com/t/sciann/shared_invite/zt-ne1f5jlx-k_dY8RGo3ZreDXwz0f~CeA" 
 
TensorFlow Version: 2.3.0 
Python Version: 3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)] 



In [3]:
import Pinn_Worker

In [4]:
valData = [['p.txt','u.txt','v.txt'],'wtunnel']


test_gridObj = SEQ.Grid(3,{'x':0,'y':1,'t':2},[[[-2,0],[0,1],[0,20]],[[0,18],[-0.5,1],[0,20]]],9)
configspecs = {
    'denspt':[3,6],
    'numNeurons':[10,100],
    'numLayers':[2,50],
    'activator':['tanh','relu','softmax','sigmoid'],
    'loss':['mae','mse'],
    'optimizer':['Adam','RMSprop','SGD','Nadam','Ftrl'],
    'batch_size':[5000,10000,15000],
    'initial_lr':[1e-4,1e-2]
}


In [5]:
# Step 1: Start a nameserver
# Every run needs a nameserver. It could be a 'static' server with a
# permanent address, but here it will be started for the local machine with the default port.
# The nameserver manages the concurrent running workers across all possible threads or clusternodes.
# Note the run_id argument. This uniquely identifies a run of any HpBandSter optimizer.

numWorkers = 1
experiment_name = 'flow_exp'
run_id = 'flow_GridSearch'

NS = hpns.NameServer(run_id=run_id, host='127.0.0.1', port=None)
NS.start()

# Step 2: Start a worker
# Now we can instantiate a worker, providing the mandatory information
# Besides the sleep_interval, we need to define the nameserver information and
# the same run_id as above. After that, we can start the worker in the background,
# where it will wait for incoming configurations to evaluate.


workers=[]
for i in range(1,numWorkers+1):
    print('Creating worker')
    w = Pinn_Worker.PINN_Worker(
    valData = valData,
    test_gridObj=test_gridObj,
    PDESystem=mySys,
    configspecs = configspecs,
    valFromFEM=True,
    experiment_name = experiment_name,
    eval_mode = 'parameter',
    valParams = {'rey':0.0025},
    eval_functionals = ['p','u','v'],
    nameserver='127.0.0.1',
    run_id=run_id,
    id=i)
    
    print('Worker created, starting it')
    w.run(background=True)
    print('Worker running, adding it to the list')
    workers.append(w)
    print('Worker added')
    


Creating worker
Creating client
Existing experiment. Recovering id.
Parsing validation data
Applying validation data to test grid
Saving PDE and config
Worker ready
Worker created, starting it
Worker running, adding it to the list
Worker added


23:14:01 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x21381d68b88; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
23:14:01 WORKER: No dispatcher found. Waiting for one to initiate contact.
23:14:01 WORKER: start listening for jobs


In [ ]:
# bohb = BOHB(  configspace = w.get_configspace(),
#               run_id = 'example1', nameserver='127.0.0.1',
#               min_budget=10, max_budget=1000
#            )
# res = bohb.run(n_iterations=25
#               )

bohb = BOHB(  configspace = w.get_configspace(),
              run_id = run_id,
              min_budget=10, max_budget=500
           )
res = bohb.run(n_iterations=5, min_n_workers=numWorkers)


23:14:25 wait_for_workers trying to get the condition
23:14:25 DISPATCHER: started the 'discover_worker' thread
23:14:25 DISPATCHER: started the 'job_runner' thread
23:14:25 DISPATCHER: Pyro daemon running on localhost:58320
23:14:25 DISPATCHER: Starting worker discovery
23:14:25 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
23:14:25 DISPATCHER: discovered new worker, hpbandster.run_flow_GridSearch.worker.DESKTOP-NKTB8UL.6332.19292
23:14:25 HBMASTER: number of workers changed to 1
23:14:25 Enough workers to start this run!
23:14:25 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
23:14:25 adjust_queue_size: lock accquired
23:14:25 HBMASTER: starting run at 1648415665.8825176
23:14:25 HBMASTER: adjusted queue size to (0, 1)
23:14:25 DISPATCHER: Finished worker discovery
23:14:25 start sampling a new configuration.
23:14:25 done sampling a new configuration.
23:14:25 DISPATCHER: Trying to submit another job.
23:14:25 HBMASTER: schedule new run for 


Total samples: 37120 
Batch size: 15000 
Total batches: 3 



23:15:25 DISPATCHER: Starting worker discovery
23:15:25 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:15:25 DISPATCHER: Finished worker discovery
23:16:25 DISPATCHER: Starting worker discovery
23:16:25 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:16:25 DISPATCHER: Finished worker discovery
23:16:48 WORKER: done with job (0, 0, 0), trying to register it.
23:16:48 WORKER: registered result for job (0, 0, 0) with dispatcher
23:16:48 DISPATCHER: job (0, 0, 0) finished
23:16:48 DISPATCHER: register_result: lock acquired
23:16:48 DISPATCHER: job (0, 0, 0) on hpbandster.run_flow_GridSearch.worker.DESKTOP-NKTB8UL.6332.19292 finished
23:16:48 job_id: (0, 0, 0)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 15000, 'denspt': 4, 'initial_lr': 0.004942500871467096, 'loss': 'mse', 'numLayers': 48, 'numNeurons': 47, 'optimizer': 'RMSprop'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: None
exception: Traceback (most recent call l


Total samples: 125280 
Batch size: 15000 
Total batches: 9 



23:17:25 DISPATCHER: Starting worker discovery
23:17:25 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:17:25 DISPATCHER: Finished worker discovery
23:18:13 WORKER: done with job (0, 0, 1), trying to register it.
23:18:13 WORKER: registered result for job (0, 0, 1) with dispatcher
23:18:13 DISPATCHER: job (0, 0, 1) finished
23:18:13 DISPATCHER: register_result: lock acquired
23:18:13 DISPATCHER: job (0, 0, 1) on hpbandster.run_flow_GridSearch.worker.DESKTOP-NKTB8UL.6332.19292 finished
23:18:13 job_id: (0, 0, 1)
kwargs: {'config': {'activator': 'tanh', 'batch_size': 15000, 'denspt': 6, 'initial_lr': 0.0011765460994350027, 'loss': 'mae', 'numLayers': 31, 'numNeurons': 49, 'optimizer': 'Ftrl'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: None
exception: Traceback (most recent call last):
  File "C:\Users\Felix\anaconda3\envs\tf_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, 

...

23:41:43 DISPATCHER: Starting worker discovery
23:41:43 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:41:43 DISPATCHER: Finished worker discovery
23:41:50 WORKER: done with job (0, 0, 18), trying to register it.
23:41:50 WORKER: registered result for job (0, 0, 18) with dispatcher
23:41:50 DISPATCHER: job (0, 0, 18) finished
23:41:50 DISPATCHER: register_result: lock acquired
23:41:50 DISPATCHER: job (0, 0, 18) on hpbandster.run_flow_GridSearch.worker.DESKTOP-NKTB8UL.6332.19292 finished
23:41:50 job_id: (0, 0, 18)
kwargs: {'config': {'activator': 'sigmoid', 'batch_size': 5000, 'denspt': 6, 'initial_lr': 0.009675537219769513, 'loss': 'mse', 'numLayers': 20, 'numNeurons': 79, 'optimizer': 'Adam'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: None
exception: Traceback (most recent call last):
  File "C:\Users\Felix\anaconda3\envs\tf_gpu\lib\site-packages\hpbandster\core\worker.py", line 206, in start_computation
    result = {'result': self.compute(*


Total samples: 68000 
Batch size: 10000 
Total batches: 7 



23:42:43 DISPATCHER: Starting worker discovery
23:42:43 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:42:43 DISPATCHER: Finished worker discovery
23:43:43 DISPATCHER: Starting worker discovery
23:43:43 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:43:43 DISPATCHER: Finished worker discovery
23:44:34 WORKER: done with job (0, 0, 19), trying to register it.
23:44:34 WORKER: registered result for job (0, 0, 19) with dispatcher
23:44:34 DISPATCHER: job (0, 0, 19) finished
23:44:34 DISPATCHER: register_result: lock acquired
23:44:34 DISPATCHER: job (0, 0, 19) on hpbandster.run_flow_GridSearch.worker.DESKTOP-NKTB8UL.6332.19292 finished
23:44:34 job_id: (0, 0, 19)
kwargs: {'config': {'activator': 'softmax', 'batch_size': 10000, 'denspt': 5, 'initial_lr': 0.0018515102268126533, 'loss': 'mse', 'numLayers': 25, 'numNeurons': 11, 'optimizer': 'RMSprop'}, 'budget': 18.51851851851852, 'working_directory': '.'}
result: None
exception: Traceback (most recent 


Total samples: 68000 
Batch size: 5000 
Total batches: 14 



23:45:43 DISPATCHER: Starting worker discovery
23:45:43 DISPATCHER: Found 1 potential workers, 1 currently in the pool.
23:45:43 DISPATCHER: Finished worker discovery


In [ ]:
np.save('HpBandster_Results',res,allow_pickle=True)
bohb.shutdown(shutdown_workers=True)
NS.shutdown()